# 第四次课作业：向量数据库与高级检索技术

本次作业旨在帮助大家深入理解：
1. 向量嵌入（Vector Embeddings）的原理及向量数据库（Vector DB）的操作。
2. 提升检索效率和效果的高级技术（如混合检索、重排序等）。

## 任务一：向量嵌入与向量数据库基础

### 1.1 环境准备
我们需要安装必要的库，包括 `sentence-transformers` 用于生成向量，以及 `chromadb` 作为我们的向量数据库。

In [1]:
%pip install sentence-transformers chromadb

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


### 1.2 生成向量嵌入
向量嵌入是将文本转化为数值向量的过程。语义相近的文本在向量空间中的距离也更近。

In [2]:
from sentence_transformers import SentenceTransformer

# 加载一个轻量级的开源模型
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = [
    "人工智能正在改变世界",
    "机器学习是人工智能的一个分支",
    "今天天气不错，适合出去玩"
]

# 编码为向量
embeddings = model.encode(sentences)

print(f"向量维度: {embeddings.shape[1]}")
for i, sent in enumerate(sentences):
    print(f"句子: {sent[:10]}... -> 向量前5位: {embeddings[i][:5]}")

/opt/homebrew/Caskroom/miniconda/base/envs/rag-project01/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/homebrew/Caskroom/miniconda/base/envs/rag-project01/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


向量维度: 384
句子: 人工智能正在改变世界... -> 向量前5位: [ 0.01836501  0.14913869 -0.00178922 -0.02026898 -0.02821331]
句子: 机器学习是人工智能的... -> 向量前5位: [ 0.05346945  0.13163942  0.00374253 -0.01207019 -0.00900904]
句子: 今天天气不错，适合出... -> 向量前5位: [0.02219923 0.08818837 0.10373423 0.02381292 0.01968082]


### 1.3 使用 ChromaDB 构建知识库
我们将使用 ChromaDB 存储这些向量并演示如何进行相似度搜索。

In [3]:
import chromadb
from chromadb.utils import embedding_functions

# 初始化客户端
client = chromadb.Client()

# 使用 sentence-transformers 作为默认嵌入函数
default_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# 创建 Collection
collection = client.create_collection(name="my_documents", embedding_function=default_ef)

# 添加文档
collection.add(
    documents=[
        "大语言模型可以处理复杂的自然语言任务",
        "向量数据库是 RAG 系统的重要组成部分",
        "北京是中国的首都"
    ],
    ids=["id1", "id2", "id3"]
)

# 执行查询
results = collection.query(
    query_texts=["什么是大模型？"],
    n_results=1
)

print("查询结果:", results['documents'])

查询结果: [['大语言模型可以处理复杂的自然语言任务']]


## 任务二：提升检索效果的技术

### 2.1 混合检索 (Hybrid Search)
混合检索结合了传统的关键字检索（BM25）和语义检索（Vector Search），以平衡精确匹配和语义理解。虽然 ChromaDB 原生支持向量检索，但在实际 RAG 中，通常会配合 Elasticsearch 或简单的 BM25 库使用。

In [5]:
# 这里演示伪代码逻辑或一个简单的 BM25 实现思路
try:
    from rank_bm25 import BM25Okapi
except ImportError:
    %pip install rank-bm25
    from rank_bm25 import BM25Okapi

corpus = [
    "大 语言 模型 是 指 具有 大量 参数 的 网络",
    "向量 数据库 支持 高效 的 相似度 搜索",
    "北京 是 一个 历史 悠久 的 城市"
]
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

query = "模型"
tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
print("BM25 分数:", doc_scores)

BM25 分数: [0.4591691 0.        0.       ]


### 2.2 重排序 (Reranking)
第一步检索（Recall）可能会召回数十个文档。为了提高精度，我们可以使用 Cross-Encoder 对召回的文档进行重新打分排序。

In [6]:
from sentence_transformers import CrossEncoder

rerank_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

query = "什么是大模型？"
passages = [
    "大模型是指拥有数十亿参数的学习网络。",
    "向量数据库可以加速搜索过程。",
    "今天我吃了一个苹果。"
]

# 构造成对输入
model_inputs = [[query, p] for p in passages]
scores = rerank_model.predict(model_inputs)

print("重排序得分:", scores)
# 得分越高表示越相关

重排序得分: [7.2941   5.21036  2.031652]


## 总结
通过本次作业，我们完成了：
1. 使用 `SentenceTransformer` 生成向量。
2. 使用 `ChromaDB` 进行向量存储和语义搜索。
3. 理解了混合检索和重排序如何进一步提升系统的 RAG 效果。